In [4]:
import torch
from transformers import CLIPImageProcessor, CLIPModel, CLIPTokenizer
from PIL import Image

device = "cuda:2"

# Load the CLIP model
model_ID = "openai/clip-vit-large-patch14"
model = CLIPModel.from_pretrained(model_ID).to(device)
preprocess = CLIPImageProcessor.from_pretrained(model_ID)

# Define a function to load an image and preprocess it for CLIP
def load_and_preprocess_image(image_path):
    # Load the image from the specified path
    image = Image.open(image_path)
    # Apply the CLIP preprocessing to the image
    image = preprocess(image, return_tensors="pt")
    # Return the preprocessed image
    return image

# Load the two images and preprocess them for CLIP
image_a = load_and_preprocess_image('./test2.png')["pixel_values"].to(device)
image_b = load_and_preprocess_image('./test1.jpg')["pixel_values"].to(device)

# Calculate the embeddings for the images using the CLIP model
# with torch.no_grad():
embedding_a = model.get_image_features(image_a)
embedding_b = model.get_image_features(image_b)

# Calculate the cosine similarity between the embeddings
similarity_score = torch.nn.functional.cosine_similarity(embedding_a, embedding_b)

# Print the similarity score
print('Similarity score:', similarity_score.item())

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


Similarity score: 0.6449080109596252


In [ ]:
optimizer = torch.optim.AdamW(model.parameters())

# Load the two images and preprocess them for CLIP
image_a = load_and_preprocess_image('./test2.png')["pixel_values"].to(device)
image_b = load_and_preprocess_image('./test1.jpg')["pixel_values"].to(device)

# Calculate the embeddings for the images using the CLIP model
# with torch.no_grad():
embedding_a = model.get_image_features(image_a)
embedding_b = model.get_image_features(image_b)

# Calculate the cosine similarity between the embeddings
similarity_score = torch.nn.functional.cosine_similarity(embedding_a, embedding_b)
similarity_score.backward()
optimizer.step()
optimizer.zero_grad()

In [13]:
print(model.vision_model(image_a)['last_hidden_state'].shape)
print(model.vision_model(image_a)['pooler_output'].shape)

torch.Size([1, 257, 1024])
torch.Size([1, 1024])


In [16]:
model.visual_projection(model.vision_model(image_a)['pooler_output']).shape

torch.Size([1, 768])

In [5]:
embedding_a = model.get_image_features(image_a)